In [ ]:
from utils import inference_validator, reasoning, feature_engineering
import pandas as pd
pd.set_option('display.max_rows', None)  # None means unlimited rows
pd.set_option('display.max_columns', None) # None means unlimited columns
pd.set_option('display.width', None)      # None means auto-detect width
pd.set_option('display.max_colwidth', None) # None means unlimited column width

In [ ]:
import pickle, json
with open('saved/nimputer.pkl', 'rb') as f:
    nimputer = pickle.load(f)

with open('saved/cimputer.pkl', 'rb') as f:
    cimputer = pickle.load(f)

with open('saved/xgb_model.pkl', 'rb') as f:
    xgb = pickle.load(f)

with open('saved/before_feature.json', 'r') as f:
    before_columns = json.load(f)


with open('saved/df_train_encoded.json', 'r') as f:
    df_train_encoded = json.load(f)
    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

user_input = {
  "ApplicationDate": "2024-02-27",
  "Age": 35,
  "AnnualIncome": 75000,
  "CreditScore": 680,
  "EmploymentStatus": "Employed",
  "EducationLevel": "Bachelor's",
  "Experience": 10,
  "LoanAmount": 15000,
  "LoanDuration": 36,
  "MaritalStatus": "Married",
  "NumberOfDependents": 2,
  "HomeOwnershipStatus": "Owned",
  "MonthlyDebtPayments": 1200,
  "CreditCardUtilizationRate": 0.3,
  "NumberOfOpenCreditLines": 5,
  "NumberOfCreditInquiries": 2,
  "DebtToIncomeRatio": 0.35,
  "BankruptcyHistory": 0,
  "LoanPurpose": "Home Renovation",
  "PreviousLoanDefaults": 0,
  "PaymentHistory": 20,
  "LengthOfCreditHistory": 12,
  "SavingsAccountBalance": 5000,
  "CheckingAccountBalance": 3000,
  "TotalAssets": 100000,
  "TotalLiabilities": 40000,
  "MonthlyIncome": 6250,
  "UtilityBillsPaymentHistory": 0.7,
  "JobTenure": 5,
  "NetWorth": 60000,
  "BaseInterestRate": 3.5,
  "InterestRate": 6.2,
  "MonthlyLoanPayment": 450,
  "TotalDebtToIncomeRatio": 0.42,
  "LoanApproved": 1,
  "RiskScore": 72
}

user_input = inference_validator(user_input)

user_df = pd.DataFrame([user_input])

# Ensure the user input has the same columns as the training data
user_df = user_df.reindex(columns=before_columns, fill_value=np.nan)
# Assuming you know which features each imputer was trained on
c_features = [f for f in cimputer.feature_names_in_ if f in user_df.columns]
n_features = [f for f in nimputer.feature_names_in_ if f in user_df.columns]


# Apply only if features exist
if c_features:
    user_df[c_features] = cimputer.transform(user_df[c_features])
if n_features:
    user_df[n_features] = nimputer.transform(user_df[n_features])


user_df = feature_engineering(user_df)
user_df = user_df.reindex(columns=df_train_encoded, fill_value=0)
prediction = xgb.predict(user_df)
prediction_proba = xgb.predict_proba(user_df)

print("Prediction:", prediction)
print(reasoning[str(prediction[0])])
print("Prediction Probability:", prediction_proba)
# print(user_processed)
# 6 → 3 → 7 → 1 → 2 → 0 → 5 → 4